## The required format for the GLAD program is:
```
First line:
    <numGivenLabels> <numLabelers> <numTasks> <Prior p(Z=1)>
  Following <numGivenLabels> lines:
    <taskId> <labelerId> <label:0|1>
```
0: Not a Disease (Includes "Phenotype")
<br>
1: Disease (Includes "Both")

I think task refers to the word in the dataset, and then labeler ID and the label. So if we have 20000 words in M2C and 5 label, the output of this program should have 100000 lines.

In [1]:
dataset_folder = 'dataset/Mark2Cure_citizen_phenotype/'

In [2]:
citizens = ['1', '2', '3', '4', '5']

In [3]:
filename = 'M2C{}-Disease_Phen_Preliminary_Run_1st_pass.txt'

In [4]:
text = ''

In [16]:
## Make vocab
vocab = {}
reverse_vocab = {}
count_vocab = dict()
path = dataset_folder + filename.format('1')
i = 0
total_lines = 0
with open(path) as f:
    lines = f.read().split('\n')
    for line in lines:
        if line:
            total_lines += 1
            word = line.split()[0]
            word = word.lower()
            c = count_vocab.get(word, 0)
            count_vocab[word] = c + 1
            word = word + '_{}'.format(i)
            if word.lower() not in vocab:
                vocab[word.lower()] = i
                reverse_vocab[i] = word.lower()
                i += 1

In [17]:
len(vocab), len(count_vocab)

(7863, 2245)

In [7]:
# make 5 arrays of text
annotations = []
for citizen in citizens:
    path = dataset_folder + filename.format(citizen)
    with open(path) as f:
        lines = f.read().split('\n')
        annotations.append(lines)

In [8]:
text = '{} {} {} {} {} {}\n'.format(len(vocab)*len(annotations), len(annotations), len(vocab), 2, 0.5, 0.5)

In [11]:
len(annotations), total_lines

(7864, 7863)

In [12]:
record = dict()
for line in range(total_lines):
    for annotator in range(len(annotations)):
        current_line = annotations[annotator][line].split()
        word = current_line[0].lower() + '_{}'.format(line)
        annotation = current_line[3]
        count = record.get(word, 0)
        if count < len(annotations):
            label = '1'
            if annotation == 'O':
                label = '0'
            text = text + '{} {} {}\n'.format(vocab[word], annotator, label)
            record[word] = count + 1

In [14]:
with open('data_phenotype_wordidx.txt', 'w') as f:
    f.write(text)

## The following code is to convert the alpha, beta and label csv to readable formats AFTER we run GLAD

### Make sure you have the proper files in the proper folder before running these hacky snippets

In [24]:
results_folder = 'Results for word_idx/'
files = ['beta.csv', 'label.csv']

In [34]:
dict_betas = {}
text = ''
with open(results_folder+'beta.csv', 'r') as f:
    lines = f.readlines()
    for line in lines:
        if line:
            arr = line.split(',')
            num = arr[0]
            if num == '# id':
                continue
            word = reverse_vocab[int(num)-1].split('_')[0]
            beta_temp = dict_betas.get(word, 0.0)
            dict_betas[word] = beta_temp + float(arr[1])
for word, _ in dict_betas.items():
    text += '{}\t{}\n'.format(word, dict_betas[word]/count_vocab[word])
with open(results_folder+'beta_word_idx.csv','w') as g:
    g.write(text)

In [35]:
text = ''
p_0 = {}
p_1 = {}
with open(results_folder+'label.csv', 'r') as f:
    lines = f.readlines()
    for line in lines:
        if line:
            arr = line.split(',')
            num = arr[0]
            if num == '# id':
                continue
            word = reverse_vocab[int(num)-1].split('_')[0]
            
            p_0_value = p_0.get(word, 0.0)
            p_0[word] = p_0_value + float(arr[1])
            
            p_1_value = p_1.get(word, 0.0)
            p_1[word] = p_1_value + float(arr[2])
            
for word, _ in p_0.items():
    text += '{}\t{}\t{}\n'.format(word, p_0[word]/count_vocab[word], p_1[word]/count_vocab[word])
    
with open(results_folder+'label_word_idx.csv','w') as g:
    g.write(text)